In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import json
import time
import re

In [45]:
driver = webdriver.Chrome()
driver.get("https://www.imdb.com")
driver.maximize_window()


In [46]:
menu = driver.find_element(By.XPATH, r"//*[@id='imdbHeader-navDrawerOpen']")

In [47]:
menu.click()
time.sleep(1)

In [48]:
link_populares = driver.find_element(By.LINK_TEXT, r'250 séries mais populares')

In [49]:
url = link_populares.get_attribute("href")
url

'https://www.imdb.com/pt/chart/toptv/?ref_=nv_tvv_250'

In [50]:
driver.get(url)

In [51]:
tag_ul = driver.find_element(By.XPATH, r'//*[@id="__next"]/main/div/div[3]/section/div/div[2]/div/ul')
lista_series = tag_ul.find_elements(By.TAG_NAME, "li")

In [52]:
type(lista_series)

list

In [ ]:
series_list = []
i=0
for serie in lista_series:
    i+=1
    print(f"{i}/{len(series_list)}", end="\r")
    titulo = str(serie.find_element(By.CLASS_NAME, "ipc-title__text").text.split(".", 1)[-1]).strip()
    link = serie.find_element(By.CLASS_NAME, "ipc-title-link-wrapper").get_attribute('href')
    ano = serie.find_element(By.XPATH, r'./div/div/div/div/div[2]/div[2]/span[1]').text
    imdb = float(serie.find_elements(By.CSS_SELECTOR, ".ipc-rating-star--imdb")[0].text.split("\n")[0].replace(",","."))
    ep = int(serie.find_element(By.XPATH, r'./div/div/div/div/div[2]/div[2]/span[2]').text.split(" ")[0])
    
    series_list.append({
         "titulo": titulo,
         "anolancamento": ano,
         "episodios": ep,
         "imdb": imdb,
         "link": link
        })

{'titulo': 'Breaking Bad', 'anolancamento': '2008–2013', 'episodios': 62, 'imdb': 9.5, 'link': 'https://www.imdb.com/pt/title/tt0903747/?ref_=chttvtp_t_1'}
{'titulo': 'Planeta Terra II', 'anolancamento': '2016', 'episodios': 6, 'imdb': 9.4, 'link': 'https://www.imdb.com/pt/title/tt5491994/?ref_=chttvtp_t_2'}
{'titulo': 'Planeta Terra', 'anolancamento': '2006', 'episodios': 11, 'imdb': 9.4, 'link': 'https://www.imdb.com/pt/title/tt0795176/?ref_=chttvtp_t_3'}
{'titulo': 'Irmãos de Guerra', 'anolancamento': '2001', 'episodios': 10, 'imdb': 9.4, 'link': 'https://www.imdb.com/pt/title/tt0185906/?ref_=chttvtp_t_4'}
{'titulo': 'Chernobyl', 'anolancamento': '2019', 'episodios': 5, 'imdb': 9.3, 'link': 'https://www.imdb.com/pt/title/tt7366338/?ref_=chttvtp_t_5'}
{'titulo': 'A Escuta', 'anolancamento': '2002–2008', 'episodios': 60, 'imdb': 9.3, 'link': 'https://www.imdb.com/pt/title/tt0306414/?ref_=chttvtp_t_6'}
{'titulo': 'Avatar: A Lenda de Aang', 'anolancamento': '2005–2008', 'episodios': 62,

In [58]:
i=0
for serie in series_list:
    i+=1
    print(f"{i}/{len(series_list)}", end="\r")
    try:
        driver.get(serie['link'])
        
        tag = driver.find_element(By.XPATH, r'//*[@id="__next"]/main/div/section/section/div[3]/section/section/div[2]/div[2]/div/div[3]/a/span/div/div[2]/div').text
        serie['popularidade'] = int(tag.replace(".",""))

        lista_elenco = driver.find_elements(By.CSS_SELECTOR, "[data-testid='title-cast-item']")
        
        elenco = [ator.find_element(By.CSS_SELECTOR, "[data-testid='title-cast-item__actor']").text for ator in lista_elenco]
        serie["atores"] = elenco

        personagens = [personagem.find_elements(By.CSS_SELECTOR, "[data-testid='cast-item-characters-link']")[0].text for personagem in lista_elenco]
        serie["personagens"] = personagens
    except:
        print("Ator ou personagem vazio.")
        continue

{'titulo': 'Breaking Bad', 'anolancamento': '2008–2013', 'episodios': 62, 'imdb': 9.5, 'link': 'https://www.imdb.com/pt/title/tt0903747/?ref_=chttvtp_t_1', 'popularidade': 26, 'atores': ['Bryan Cranston', 'Aaron Paul', 'Anna Gunn', 'Betsy Brandt', 'RJ Mitte', 'Dean Norris', 'Bob Odenkirk', 'Steven Michael Quezada', 'Jonathan Banks', 'Giancarlo Esposito', 'Charles Baker', 'Jesse Plemons', 'Christopher Cousins', 'Laura Fraser', 'Matt Jones', 'Michael Shamus Wiles', 'Lavell Crawford', 'Ray Campbell'], 'personagens': ['Walter White', 'Jesse Pinkman', 'Skyler White', 'Marie Schrader', 'Walter White, Jr.', 'Hank Schrader', 'Saul Goodman', 'Steven Gomez', 'Mike Ehrmantraut', 'Gus Fring', 'Skinny Pete', 'Todd', 'Ted Beneke', 'Lydia Rodarte-Quayle', 'Badger', 'ASAC George Merkert', 'Huell', 'Tyrus Kitt']}
{'titulo': 'Planeta Terra II', 'anolancamento': '2016', 'episodios': 6, 'imdb': 9.4, 'link': 'https://www.imdb.com/pt/title/tt5491994/?ref_=chttvtp_t_2', 'popularidade': 1126, 'atores': ['Davi

In [ ]:
with open("filmes.json", "w", encoding="utf-8") as arquivo:
    for serie in series_list:
        json.dump(serie, arquivo, ensure_ascii=False, indent=4)

In [ ]:
driver.quit()